- Why x did not change?
- We can pass an argument to a function: 
  - by value
  - by reference

The IncrementByOne() passes the argument (x) by value:
- ▶ a copy of the variable (x) is passed to the function
- ▶ we’re changing the copy; the original value remains unchanged

- When we want a function to use the same variable (object) inside the function, we pass the argument by reference.
- A reference is an alias (another name) for a variable (object).


//
// Created by Tobias Rodriguez del Pozo on 3/9/23.
//

#ifndef ASSIGNMENT6_BINOMIALTREEPRICER_H
#define ASSIGNMENT6_BINOMIALTREEPRICER_H

#include <vector>
#include "Option.h"

using namespace std;

struct BinomialTreePricerNode {
    double S;
    double payoff;
    // BinomialTreePricerNode *left;
    // BinomialTreePricerNode *right;
};

class BinomialTreePricer {
public:
    // Constructor
    BinomialTreePricer(double S, double r, double sigma, double T, unsigned long N);

    // Copy Constructor
    BinomialTreePricer(const BinomialTreePricer &BinomialTreePricer);

    // Destructor
    ~BinomialTreePricer();

    // Getters and Setters. Note that we don't need getters for
    // the precalculated variables. Additionally, we update the
    // tree upon setting a new value that impacts the tree.
    double getS0() const;
    void setS0(double S0);

    double getR() const;
    void setR(double r);

    double getSigma() const;
    void setSigma(double sigma);

    double getT() const;
    void setT(double T);

    unsigned long getN() const;
    void setN(unsigned long N);

    // Assignment Operator
    BinomialTreePricer &operator=(const BinomialTreePricer &BinomialTreePricer);

    //Pricing function
    double price(Option &option);

private:
    void buildTree();
    // Member Variables
    double S0_;
    double r_;
    double sigma_;
    double T_;
    unsigned long N_;

    // Member Variables for precalculations
    double nu_;
    double dt_;
    double disc_t;
    double sqrt_dt_;

    using Tree = vector<vector<BinomialTreePricerNode>>;
    Tree tree_;

};


#endif //ASSIGNMENT6_BINOMIALTREEPRICER_H

//
// Created by Tobias Rodriguez del Pozo on 3/9/23.
//

#include "BinomialTreePricer.h"
#include <cmath>

using namespace std;

// Constructor
BinomialTreePricer::BinomialTreePricer(double S, double r, double sigma, double T, unsigned long N) :
        S0_(S),
        r_(r),
        sigma_(sigma),
        T_(T),
        N_👎
{
    nu_ = r_ - 0.5 * sigma_ * sigma_;
    dt_ = T_ / N_;
    disc_t = exp(-r_ * dt_);
    sqrt_dt_ = sqrt(dt_);
    buildTree();
}

// Copy Constructor
BinomialTreePricer::BinomialTreePricer(const BinomialTreePricer &BinomialTreePricer) :
        S0_(BinomialTreePricer.S0_),
        r_(BinomialTreePricer.r_),
        sigma_(BinomialTreePricer.sigma_),
        T_(BinomialTreePricer.T_),
        N_(BinomialTreePricer.N_),
        nu_(BinomialTreePricer.nu_),
        dt_(BinomialTreePricer.dt_),
        disc_t(BinomialTreePricer.disc_t),
        sqrt_dt_(BinomialTreePricer.sqrt_dt_),
        tree_(BinomialTreePricer.tree_)
{}

// Destructor
BinomialTreePricer::~BinomialTreePricer() = default;

// Assignment Operator
BinomialTreePricer &BinomialTreePricer::operator=(const BinomialTreePricer &BinomialTreePricer) {
    S0_ = BinomialTreePricer.S0_;
    r_ = BinomialTreePricer.r_;
    sigma_ = BinomialTreePricer.sigma_;
    T_ = BinomialTreePricer.T_;
    N_ = BinomialTreePricer.N_;
    nu_ = BinomialTreePricer.nu_;
    dt_ = BinomialTreePricer.dt_;
    disc_t = BinomialTreePricer.disc_t;
    sqrt_dt_ = BinomialTreePricer.sqrt_dt_;
    tree_ = BinomialTreePricer.tree_;
    return *this;
}

// Getters and setters.
double BinomialTreePricer::getS0() const {
    return S0_;
}

// Note that when we change the value of S0,
// we need to rebuild the tree.
void BinomialTreePricer::setS0(double S0) {
    S0_ = S0;
    buildTree();
}

double BinomialTreePricer::getR() const {
    return r_;
}

// Note that when we change the value of r,
// we need to rebuild the tree and also update
// the value of nu.
void BinomialTreePricer::setR(double r) {
    r_ = r;
    nu_ = r_ - 0.5 * sigma_ * sigma_;
    buildTree();
}

double BinomialTreePricer::getSigma() const {
    return sigma_;
}

// Note that when we change the value of sigma,
// we need to rebuild the tree and also update
// the value of nu.
void BinomialTreePricer::setSigma(double sigma) {
    sigma_ = sigma;
    nu_ = r_ - 0.5 * sigma_ * sigma_;
    buildTree();
}

double BinomialTreePricer::getT() const {
    return T_;
}

// Note that when we change the value of T,
// we need to rebuild the tree and also update
// the value of dt and sqrt_dt.
void BinomialTreePricer::setT(double T) {
    T_ = T;
    dt_ = T_ / N_;
    sqrt_dt_ = sqrt(dt_);
    buildTree();
}

unsigned long BinomialTreePricer::getN() const {
    return N_;
}

// Note that when we change the value of N,
// we need to rebuild the tree and also update
// the value of dt and sqrt_dt.
void BinomialTreePricer::setN(unsigned long N) {
    N_ = N;
    dt_ = T_ / N_;
    sqrt_dt_ = sqrt(dt_);
    buildTree();
}

// Build the tree of stock paths.
void BinomialTreePricer::buildTree() {
    // Initialize tree
    tree_.resize(N_ + 1);
    for (int i = 0; i < N_ + 1; i++) {
        tree_[i].resize(i + 1);
    }

    // Set the first node to be the price of
    // the stock now.
    tree_[0][0].S = S0_;

    // Specify the "up" and "down" state according
    // to Jarrow-Rudd Binomial Model.
    double u = exp(nu_ * dt_ + sigma_ * sqrt_dt_);
    double d = exp(nu_ * dt_ - sigma_ * sqrt_dt_);

    // Fill in the tree. Each node is the price of
    // the stock given a certain amount of time steps,
    // and a given amount of "up" vs. "down" states.
    for (int i = 0; i < N_ + 1; i++) {
        for (int j = 0; j < i + 1; j++) {
            tree_[i][j].S = S0_ * pow(u, j) * pow(d, i - j);
        }
    }
}

// Pricing function
double BinomialTreePricer::price(Option &option) {
    // Calculate the expiration payoff for each node.
    for (int j = 0; j < N_ + 1; j++) {
        tree_[N_][j].payoff = option.expirationPayoff(tree_[N_][j].S);
    }

    // Loop backwards through the tree, calculating the intermediate payoff
    // for each node in the tree. The intermediate payoff is the discounted
    // expected payoff of the next time step for European options. And the
    // maximum of said payoff and the early exercise payoff for American
    // and other path dependent options.
    for (int i = N_ - 1; i >= 0; i--) {
        for (int j = 0; j < i + 1; j++) {
            double discPayoff = disc_t * (0.5 * tree_[i + 1][j].payoff + 0.5 * tree_[i + 1][j + 1].payoff);
            double St = tree_[i][j].S;
            tree_[i][j].payoff = option.intermediatePayoff(St, discPayoff);
        }
    }
    // Return the price of the option.
    return tree_[0][0].payoff;
}

- Solution is to pass by const-reference
- ▶ Solution is to pass by const-reference: int DotProduct(const BigMatrix& m1,
          const BigMatrix& m2)
   {
       //dot product using m1 and m2
       //cannot change m1 and m2
}
- ▶ Now, we cannot intentionally or accidentally change m1 or m2 in DotProduct().
- ▶ We’re using type safety to write ”correct” code.
- ▶ Improves readability/clarity: using const reference clearly shows our intention for using a reference.

- 1. What is a reference?
  - A reference is a type of pointer that is declared with the & operator.
- 2. Why would you use a reference?
  - We use references to avoid copying large objects. Also, we use references to pass objects to functions without copying them.

- ▶ We use use two important operators with pointers: 
- ▶ & : address-of operator
- ▶ * : dereference operator

- what's the difference between a reference and a pointer?
  - The main difference is that a reference is an alias for an existing object, whereas a pointer is an object in its own right. We can change the value of a pointer, but we cannot change the value of a reference.
  - Similarities:

Both pointers and references allow indirect access to the memory locations of variables or objects.

Both pointers and references can be used to pass arguments to functions by reference instead of by value, allowing changes made to the arguments within the function to affect the original variables or objects.

Both pointers and references can be used to create dynamic data structures, such as linked lists, trees, and graphs.

Differences:

Syntax: Pointers are declared using the * symbol, while references are declared using the & symbol.

Nullability: Pointers can be assigned a null value, which means they don't point to any valid memory location. References, on the other hand, must always refer to a valid object or variable and cannot be null.

Assignment: Pointers can be reassigned to point to different memory locations, whereas references cannot be reassigned to refer to a different object or variable after initialization.

Pointer arithmetic: Pointers can be incremented or decremented to point to different memory locations, and pointer arithmetic can be used to iterate through arrays or manipulate memory. References do not support pointer arithmetic.

Type checking: Pointers can be declared to point to any data type, including void. References, on the other hand, must be initialized to refer to an object or variable of the same type.

Memory management: Pointers require explicit memory management, meaning that the programmer is responsible for allocating and deallocating memory for the pointer. References, on the other hand, are automatically managed by the language's garbage collector.

Overall, pointers and references are similar in their ability to provide indirect access to memory locations, but differ in their syntax, nullability, assignment, type checking, and memory management.

static int count;
};

int MyClass::count = 0;

int main() {
    MyClass obj1;
    MyClass obj2;
    std::cout << "Count: " << MyClass::getCount() << std::endl;
    MyClass obj3(obj2);
    std::cout << "Count: " << MyClass::getCount() << std::endl;
    obj1 = obj2;
    std::cout << "Count: " << MyClass::getCount() << std::endl;
    return 0;
}

//Operator overloading

Main Concepts - Roadmap: 
- 1. Classes and Objects
  - Classes and objects are the two main aspects of object-oriented programming. A class is a user-defined data type, which holds its own data members and member functions, which can be accessed and used by creating an instance of that class, i.e., an object.
- 2. Data Abstraction and Encapsulation
  - Data abstraction refers to providing only essential information about the data to the outside world, hiding the background details or implementation. Encapsulation refers to the wrapping up of data under a single unit. It is the mechanism that binds together code and the data it manipulates. An object is an example of encapsulation as it contains both data and functions. Public and private access specifiers are used to achieve encapsulation in C++.
  - Encapsulation refers to combining data and functions inside a class so that data is only accessed through the functions of the class.
- 3. Inheritance
    - Inheritance is a way to form new classes using classes that have already been defined. The newly formed classes are called derived classes, and the classes that we derive from are called base classes. Using inheritance, we can reuse the code functionality of the base class. It represents the IS-A relationship which is also known as a parent-child relationship.
- 4. Polymorphism
  - Polymorphism means "many forms", and it occurs when we have many classes that are related to each other by inheritance. Like we specified in the previous chapter; inheritance lets us inherit attributes and methods from another class. Polymorphism uses those methods to perform different tasks. This allows us to perform a single action in different ways. For example, think of a superclass called Animal that has a method called animalSound(). Subclasses of Animals could be Pigs, Cats, Dogs, Birds - And they also have their own implementation of an animal sound (the pig oinks, and the cat meows, etc.): The pig says: wee wee, the cat says: meow meow.


1. public: anyone can access a public member (data/function)
of a class.
2. private: only the class and friend functions (week 6) can
access private members.
1. protected: only the class and friend functions can access

Let’s write two constructors for illustration:
1. This constructor does not take any arguments:
Currency();
2. This one takes 2 arguments: Currency(string symbol, double rate);

- ▶ struct: Members have public protection level by default. 
- ▶ class: Members have private protection level by default.

- ▶ What are include guards?
  - Include guards are a way to prevent a header file from being included more than once in a single compilation unit. This is done by defining a macro in the header file, and then checking if the macro is already defined before including the header file. If the macro is already defined, the header file is not included again.

There are more:
1. copy constructor
   1. A copy constructor is a member function which initializes an object using another object of the same class. Give and example code-
   
2. assignment operator
   1. An assignment operator is a special type of function in C++ which is used to copy the value of one object to another already initialized object of the same class. The assignment operator is overloaded by using the operator keyword. The syntax of the assignment operator is:

In [ ]:
class Line {
    public:
        int getLength( void );
        Line( int len );             // simple constructor
        Line( const Line &obj);      // copy constructor
        ~Line();                     // destructor
        
    private:
        int *ptr;
};

#Example of assignment operator
# Path: final_prep.ipynb
class Line {
    public:
        int getLength( void );
        Line( int len );             // simple constructor
        Line( const Line &obj);      // copy constructor
        ~Line();                     // destructor
        Line& operator = (const Line &obj); // assignment operator
        
    private:
        int *ptr;
};

- ▶ Every non static1 class member function has access to an implicit pointer; name of the pointer is "this".
- ▶ The "this" pointer is initialized with the object’s own address.

In [ ]:
class Person {
private:
  std::string name;
  int age;

public:
  Person(std::string name, int age) {
    this->name = name;
    this->age = age;
  }

  void printInfo() {
    std::cout << "Name: " << this->name << ", Age: " << this->age << std::endl;
  }
};

int main() {
  Person p("John", 30);
  p.printInfo();
  return 0;
}

Static Members
▶ We can also associate a member/members (data/function) with the class, not with individual objects.
▶ We use static keyword to associate a member with the class.
▶ Suppose, we want to write a Counter class to keep track of some value.
▶ Also, suppose, we want multiple counters to keep track of the same value.
▶ we’re trying to raise money for a charity ▶ each one of us (one object) raises money ▶ every dollar goes to one bank account
▶ We could use static data member (bank account) here.
A static data member cannot be accessed directly using a non-static member function.
▶ Two member functions have to be static.
   class Counter
   {
   public:
      static int GetCount();
      static void Increment();
   private:
      static int count_;
};

In [ ]:
#We can explicitly ask the compiler to generate default constructor:
   class CurrencyFactory
   {
   public:
      CurrencyFactory() = default;
      Currency GetCurrency(int currencyType);
   };

### Automatic Objects
- ▶ Objects we’ve looked at so far:
- ▶ created at declaration
- ▶ destroyed when they go out of scope (i.e. exit the block of
code)
- ▶ lifetime directly tied to the scope
- ▶ They are called automatic objects.
- ▶ Easy to use: Created and destroyed automatically.

## Free Store Objects
- ▶ What if we need to manage the lifetime of the objects?
  - destroy an object when we no longer need it
  - need an object to live beyond its scope
- ▶ We can use free-store (dynamic) objects.
- The operator new:
- ▶ uses a constructor to create an object ▶ returns the address of the object
       Currency* c = new Currency("USD", 1.0);
- ▶ This object is not destroyed until we delete it, using the operator delete.
delete c;
- If we don’t/forget to delete a free-store object, we have a memory leak.
- We use the dereference operator (*) or the -> operator to access - free-store members:
  Currency* c = new Currency("EUR", 0.9494 );
  (*c).GetSymbol();
  (*c).SetExchangeRate(0.95);
- ▶ Alternatively, we could write the same code above as: c->GetSymbol();
c->SetExchangeRate(0.95);
- ▶ Common practice is to use -> with free-store objects.

##We use the destructor to delete the free store objects:
   CurrencyFactory:: ̃CurrencyFactory()
   {
     for (int i=0; i<5; ++i)
     {
         delete currencies_[i];
     }
}